In [1]:
import mgd
import os
import pandas as pd
import torchaudio
import torchaudio.transforms as transforms
import utils
import cnn
import torch.nn as nn
import torch.optim as optim
import s


In [2]:
AUDIO_DIR = os.path.expanduser("/Users/noramendoza/Desktop/DL_project/data/fma_small")
TRACKS_FILE = utils.load("/Users/noramendoza/Desktop/DL_project/TRACKS.csv")
TRACKS_FILE.head()

,track_id,genre_top
0,2,Hip-Hop
2,5,Hip-Hop
3,10,Pop
15,140,Folk
16,141,Folk


Create dataset containing *spectrograms* of song tracks and their *labels*

In [3]:
music_data = mgd.MusicGenreDataset(AUDIO_DIR, TRACKS_FILE)
print(f"There are {len(music_data)} samples in the dataset.")

There are 8000 samples in the dataset.


In [4]:
X = []
y = []

for D, slice_list, label,  in music_data: 
    for i in slice_list: 
        X.append(i)
        y.append(label)

An error occurred:  Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenj

TypeError: cannot unpack non-iterable NoneType object

In [13]:
from sklearn.preprocessing import LabelEncoder
import torch

# Create a label encoder
encoder = LabelEncoder()

# Fit the encoder on your labels and transform the labels
y_numeric = encoder.fit_transform(y)

# Now you can create a tensor from y_numeric
y = torch.tensor(y_numeric)

In [15]:
X[1]

tensor([[-49.2688, -49.2109, -51.2554,  ..., -50.0266, -43.7841, -56.2814],
        [-40.8230, -37.1824, -37.1904,  ..., -56.4383, -42.9470, -39.5480],
        [-37.7960, -33.4552, -33.4437,  ..., -48.7048, -43.6254, -44.2777],
        ...,
        [-53.9159, -48.6199, -43.5444,  ..., -42.8233, -56.0853, -51.3020],
        [-68.1076, -61.0013, -53.3235,  ..., -42.3674, -48.5893, -55.7945],
        [-67.6845, -71.4345, -55.2422,  ..., -45.7191, -49.1365, -49.9468]])

In [16]:
X[1].shape

torch.Size([128, 128])

In [17]:
len(X)

714360

In [18]:
len(y)

714360

In [19]:
from sklearn.model_selection import train_test_split

# First split the data into a training set and a temporary set using an 80-20 split.
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Then split the temporary set into validation and testing sets using a 50-50 split.
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [23]:
X_train[1].shape

torch.Size([128, 128])

In [24]:
type(y)

torch.Tensor

In [25]:
from torch.utils.data import DataLoader
import s

data = X_train # Your data (list of tensors)

# Define DataLoader parameters
batch_size = 32
shuffle = True
num_workers = 4

# Create an instance of your custom dataset
train_custom_dataset = s.final_dataset(data, y_train)
test_dataset = s.final_dataset(X_test, y_test)

# Create a DataLoader
train_dataloader = DataLoader(train_custom_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

In [26]:
import model
model.train_model(train_dataloader, test_dataloader)

: 

: 